In [1]:
# 用于验证使用平台读取本地log文件并存入es的demo
# 创建SparkSession会话，并连接es
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
import os

spark = SparkSession.builder \
    .appName("NetFlowReader") \
    .enableHiveSupport() \
    .getOrCreate() \

es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.nodes","elksj-elasticsearch:9200")
)

In [43]:
# 导入zat,将log转换为Dataframe
import zat
from zat import log_to_sparkdf

# 打印版本
print('ZAT: {:s}'.format(zat.__version__))
print('Pandas: {:s}'.format(pd.__version__))

ZAT: 0.4.4
Pandas: 1.0.4


In [44]:
# 使用zat中的log_to_sparkdf类直接读取log
spark_it = log_to_sparkdf.LogToSparkDF(spark)
http_spdf = spark_it.create_dataframe('http.log')

In [45]:
http_spdf.head()

Row(ts=datetime.datetime(2013, 9, 15, 23, 45, 50, 227456), uid='CyIaMO7IheOh38Zsi', id_orig_h='192.168.33.10', id_orig_p=1031, id_resp_h='54.245.228.191', id_resp_p=80, trans_depth=1, method='GET', host='guyspy.com', uri='/', referrer='-', user_agent='Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)', request_body_len=0, response_body_len=184, status_code=301, status_msg='Moved Permanently', info_code=0, info_msg='-', filename='-', tags='(empty)', username='-', password='-', proxied='-', orig_fuids='-', orig_mime_types='-', resp_fuids='Fnjq3r4R0VGmHVWiN5', resp_mime_types='text/html')

In [46]:
# 验证转换结果
http_pddf = http_spdf.toPandas()
http_pddf.head()

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,trans_depth,method,host,uri,...,info_msg,filename,tags,username,password,proxied,orig_fuids,orig_mime_types,resp_fuids,resp_mime_types
0,2013-09-15 23:45:50.227456,CyIaMO7IheOh38Zsi,192.168.33.10,1031,54.245.228.191,80,1,GET,guyspy.com,/,...,-,-,(empty),-,-,-,-,-,Fnjq3r4R0VGmHVWiN5,text/html
1,2013-09-15 23:45:50.227456,CoyZrY2g74UvMMgp4a,192.168.33.10,1032,54.245.228.191,80,1,GET,www.guyspy.com,/,...,-,-,(empty),-,-,-,-,-,FCQ5aX37YzsjAKpcv8,text/html
2,2013-09-15 23:45:50.227456,CoyZrY2g74UvMMgp4a,192.168.33.10,1032,54.245.228.191,80,2,GET,www.guyspy.com,/wp-content/plugins/slider-pro/css/advanced-sl...,...,-,-,(empty),-,-,-,-,-,FD9Xu815Hwui3sniSf,text/html
3,2013-09-15 23:45:50.227456,CiCKTz4e0fkYYazBS3,192.168.33.10,1040,54.245.228.191,80,1,GET,www.guyspy.com,/wp-content/plugins/contact-form-7/includes/cs...,...,-,-,(empty),-,-,-,-,-,FMZXWm1yCdsCAU3K9d,text/plain
4,2013-09-15 23:45:50.227456,C1YBkC1uuO9bzndRvh,192.168.33.10,1041,54.245.228.191,80,1,GET,www.guyspy.com,/wp-content/plugins/slider-pro/css/slider/adva...,...,-,-,(empty),-,-,-,-,-,FA4NM039Rf9Y8Sn2Rh,text/plain


In [47]:
# Store to elasticsearch把Spark DF存到es
(http_spdf.write.format('org.elasticsearch.spark.sql')
              .option("es.nodes","elksj-elasticsearch:9200")
              .option('es.resource', '%s/%s' % ('http_log', 'http'))
              .save())

Py4JJavaError: An error occurred while calling o193.save.
: org.elasticsearch.hadoop.EsHadoopIllegalStateException: SaveMode is set to ErrorIfExists and index http_log/http exists and contains data. Consider changing the SaveMode
	at org.elasticsearch.spark.sql.DefaultSource.createRelation(DefaultSource.scala:111)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.sca

In [40]:
# 测试从es拿数据
testDF = es_reader.load("http_log")
# flowDF.describe( ).show()
testDF.show()

+--------+--------------------+-------------+---------+---------------+---------+---------+--------+------+----------+---------------+--------+-------+--------------------+----------------+------------------+---------------+-----------------+-----------+-----------------+-------+-----------+-------------+------------------+--------------------+--------------------+--------+
|filename|                host|    id_orig_h|id_orig_p|      id_resp_h|id_resp_p|info_code|info_msg|method|orig_fuids|orig_mime_types|password|proxied|            referrer|request_body_len|        resp_fuids|resp_mime_types|response_body_len|status_code|       status_msg|   tags|trans_depth|           ts|               uid|                 uri|          user_agent|username|
+--------+--------------------+-------------+---------+---------------+---------+---------+--------+------+----------+---------------+--------+-------+--------------------+----------------+------------------+---------------+-----------------+----

In [41]:
flowpdDF = testDF.toPandas()
flowpdDF.head()

,filename,host,id_orig_h,id_orig_p,id_resp_h,id_resp_p,info_code,info_msg,method,orig_fuids,...,response_body_len,status_code,status_msg,tags,trans_depth,ts,uid,uri,user_agent,username
0,-,guyspy.com,192.168.33.10,1031,54.245.228.191,80,0,-,GET,-,...,184,301,Moved Permanently,(empty),1,1379288750227,CyIaMO7IheOh38Zsi,/,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,-
1,-,www.guyspy.com,192.168.33.10,1032,54.245.228.191,80,0,-,GET,-,...,100631,200,OK,(empty),1,1379288750227,CoyZrY2g74UvMMgp4a,/,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,-
2,-,www.guyspy.com,192.168.33.10,1032,54.245.228.191,80,0,-,GET,-,...,55817,404,Not Found,(empty),2,1379288750227,CoyZrY2g74UvMMgp4a,/wp-content/plugins/slider-pro/css/advanced-sl...,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,-
3,-,www.guyspy.com,192.168.33.10,1040,54.245.228.191,80,0,-,GET,-,...,887,200,OK,(empty),1,1379288750227,CiCKTz4e0fkYYazBS3,/wp-content/plugins/contact-form-7/includes/cs...,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,-
4,-,www.guyspy.com,192.168.33.10,1041,54.245.228.191,80,0,-,GET,-,...,10068,200,OK,(empty),1,1379288750227,C1YBkC1uuO9bzndRvh,/wp-content/plugins/slider-pro/css/slider/adva...,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,-


In [42]:
#flowDF = spark.createDataFrame(flowpdDF)
#flowDF.head()

Row(filename='-', host='guyspy.com', id_orig_h='192.168.33.10', id_orig_p=1031, id_resp_h='54.245.228.191', id_resp_p=80, info_code=0, info_msg='-', method='GET', orig_fuids='-', orig_mime_types='-', password='-', proxied='-', referrer='-', request_body_len=0, resp_fuids='Fnjq3r4R0VGmHVWiN5', resp_mime_types='text/html', response_body_len=184, status_code=301, status_msg='Moved Permanently', tags='(empty)', trans_depth=1, ts=1379288750227, uid='CyIaMO7IheOh38Zsi', uri='/', user_agent='Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)', username='-')